<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2022/ex12noteC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex12noteC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

----
## 階層型クラスタリングの実験
----

独立行政法人統計センターが作成している [教育用標準データセット(SSDSE)](https://www.nstac.go.jp/use/literacy/ssdse/) の中の「SSDSE-家計消費」というデータを使って，階層型クラスタリングの実験をやってみましょう．

「SSDSE-家計消費」は，47都道府県の県庁所在市で調査された家計消費に関する226項目の数値から成るデータです．「二人以上の世帯の1世帯あたりの焼き肉（外食）への年間支出金額」みたいな数値が含まれています．

データの詳細はこちら: https://www.nstac.go.jp/sys/files//kaisetsu-C-2022.pdf

この notebook の作成にあたっては，データの前処理の部分でこちらを参考にさせていただきました： https://studiolab.sagemaker.aws/import/github/icoxfog417/mlnote-note/blob/main/notebooks/chapter10_answer.ipynb

---
### データの入手と前処理

SSDSEのサイトから CSV ファイルを入手できます．URL を直接指定して Pandas の DaraFrame として読み込みます．



In [ ]:
URL = 'https://www.nstac.go.jp/sys/files//SSDSE-C-2022.csv'
df_raw = pd.read_csv(URL, encoding='cp932') # 文字コードが UTF8 ではなく ShiftJIS なので，encoding オプションを指定して変換
df_raw.head(10) # 最初の10行を表示

扱いやすいように DaraFrame を加工します．

In [ ]:
# df_raw の見出し行に記されたコードと，1行目に記されたその項目名との対応表を作り，項目名の行を削除
code = df_raw.iloc[0, :]
print('##### コードと項目名の対応')
print(code)
df = df_raw.drop(0, axis=0)

# 'SSDSE-C-2022'列の都市コードと'Prefecture'列の都道府県名，'City'列の県庁所在地名を整理
city = df.iloc[:, :3]
print()
print('##### 都市コード，都道府県名，県庁所在地名')
print(city.head(5))
df = df.drop(['Prefecture', 'City'], axis=1)
df = df.set_index('SSDSE-C-2022')

df = df.drop('R00000', axis=0) # 'R00000'列は「全国」のデータなのでこれを削除
df = df.astype('float32') # 値を浮動小数点型に変換

print()
df.head(5)

ここでは，各種の「外食」での支出金額のみを特徴として取り出します．13個の特徴量があります．

In [ ]:
# 一人あたりの外食支出金額のデータを作成
df_feature = pd.DataFrame()

# コードが 'LB12' で始まる列が，様々な品目の外食支出額．ただし，1世帯あたりの金額
columns = {}
for c in df.columns:
    if c.startswith('LB12') and c != 'LB12':
        print(c, code[c])
        columns[c] = code[c]
        df_feature[c] = df[c]

# 列名をコードから項目名に変更
df_feature = df_feature.rename(columns=columns)

# 都道府県ごとの平均世帯人数で割って一人あたりの金額にする
df_feature = df_feature.divide(df['LA03'], axis=0)

# DataFrame 表示時の小数部の表示桁数
pd.options.display.precision = 1

df_feature.head(5)

いくつかの府県を選んで特徴量を表示させてみましょう．

In [ ]:
# 大津，京都，大阪，神戸，奈良，名古屋，高松
df_hoge = df_feature.loc[['R25201', 'R26100', 'R27100', 'R28100', 'R29201', 'R23100', 'R37201'], :].copy()
city_hoge = city.set_index('SSDSE-C-2022').loc[df_hoge.index]
df_hoge['Prefecture'] = city_hoge['Prefecture']
df_hoge['City'] = city_hoge['City']

df_hoge[['Prefecture', 'City'] + list(df_feature.columns)]

値が平均 0 分散1 になるようにスケーリングして，NumPy の配列とします．
スケーリングには，[sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) クラスを用いてます．

In [ ]:
from sklearn.preprocessing import StandardScaler

# 平均 0 分散 1 にスケーリング
X = StandardScaler().fit_transform(df_feature)
print(X.shape)
print(X[:5, :]) # 最初の5つのデータ

---
### 階層型クラスタリングしてみよう



上記の配列 `X` を学習データとして，階層型クラスタリングを実行します．
[scipy.cluster.hiearchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) でクラスタリングして，
[scipy.cluster.hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) でデンドログラムを描きます．
データ点間の距離はユークリッド距離，クラス間の距離は Ward 法で測ります．

In [ ]:
import scipy.cluster.hierarchy as hierarchy

link = hierarchy.linkage(X, method='ward', metric='euclidean')
fig, ax = plt.subplots(facecolor="white", figsize=(12, 6))
hierarchy.dendrogram(link, distance_sort='descending', show_leaf_counts=True, leaf_font_size=16, ax=ax)
plt.show()

以下のセルを実行すると，`threshold` で指定した値をクラスタ間距離のしきい値としてデータをクラスタに分けた結果を表示します． `label` がクラスタの番号に相当します（0ではなく1からはじまる番号）．

In [ ]:
threshold = 6.0 # クラスタ間距離のしきい値

label = hierarchy.fcluster(link, t=threshold, criterion='distance')
df_result = city.iloc[1:, :].copy()
df_result['label'] = label
df_result['idx'] = np.arange(47, dtype=int)
df_result = df_result.set_index('idx')
df_result = df_result.sort_values('label')
df_result[['label','Prefecture', 'City']]

#### ★★★ やってみよう ★★★

- 名古屋市（上記の表やデンドログラムで22番）とよく似てるのはどこだろう？ 2つ挙げよう．
- デンドログラムを見て，クラスタの数がそれぞれ 4, 6, 9 となるようなしきい値を選ぼう．それぞれのしきい値設定でクラスタリングした結果を観察しよう．
    - 関西5府県（滋賀，京都，大阪，兵庫，奈良）の県庁所在市の値は，互いに似てる？ 似てない？ それぞれのクラスタ数の条件で同じクラスタに入っているかどうか見てみよう．
- クラスタ数が 9 以上のときは，一つのデータが孤立してそれだけで一つのクラスタとなるケースが出てきます．どこ？ そこは何が特徴的なのだろう？

----
## 主成分分析 - 前編 の内容に関する演習
----
というかほぼ線形代数の復習...



---
### 計算問題

**［問1］**$\mathbf{w}_1 = (1, 0, 1), \mathbf{x}_1 = (1, 2, 3)$ のとき，次の値を求めなさい．
1. $\mathbf{w}_1\cdot\mathbf{x}_1$
1. $\Vert\mathbf{w}_1\Vert$ および $\Vert\mathbf{x}_1\Vert$

次のセルを実行すると，答えを確認できます（数値的に計算した結果なので誤差を含みますが）．

以下では， $\Vert\mathbf{x}_1\Vert^2 = \mathbf{x}_1 \cdot \mathbf{x}_1 $ という性質を利用して $\Vert\mathbf{x}_1\Vert$ を計算してます．

In [ ]:
w1 = np.array([1.0, 0.0, 1.0])
x1 = np.array([1.0, 2.0, 3.0])
print('w1 = ', w1)
print('x1 = ', x1)
print('w1 と x1 の内積 = ', w1 @ x1)
print('|w1| = ', np.sqrt(w1 @ w1))
print('|x1| = ', np.sqrt(x1 @ x1))

**［問2］** $\mathbf{w}_2 = (0, 1, -1), \mathbf{x}_2 = (4, 5, 6)$ のとき，次の値を求めなさい．$\mathbf{w}_1, \mathbf{x}_1$ は問1のものを使ってください．以下，前の問題で出てきた記号を説明せずに使ってる場合があります．
1. $\mathbf{w}_1\cdot\mathbf{x}_2$
1. $\mathbf{w}_2\cdot\mathbf{x}_1$
1. $\mathbf{w}_2\cdot\mathbf{x}_2$


In [ ]:
w2 = np.array([0.0, 1.0, -1.0])
x2 = np.array([4.0, 5.0, 6.0])
print('w2 = ', w2)
print('x2 = ', x2)
print('w1 と x2 の内積 = ', w1 @ x2)
print('w2 と x1 の内積 = ', w2 @ x1)
print('w2 と x2 の内積 = ', w2 @ x2)

**［問3］** 次のセルを実行すると表示されるような，3行4列の行列を $X$ とします．
この行列を列ごとに眺めると，第1列は $\mathbf{x}_1$ と等しく，第2列は $\mathbf{x}_2$ と等しいことに注意しましょう．

In [ ]:
X = np.arange(1, 13).reshape((4, 3)).T
X

また，次のような 2行3列の行列を $W$ とします．こちらは，行ごとに眺めると，第1行が $\mathbf{w}_1$ と等しく，第2行が $\mathbf{w}_2$ と等しくなっています．

In [ ]:
W = np.vstack((w1, w2))
W

このとき，$Y = WX$ を手計算で求めなさい．

次のセルを実行すると，答えを確認できます．

In [ ]:
Y = W @ X
Y

結果を見ると，$Y$ の1列目には，$\mathbf{w}_1\cdot\mathbf{x}_1$ と $\mathbf{w}_2\cdot\mathbf{x}_1$ の値が入っており，2列目には，$\mathbf{w}_1\cdot\mathbf{x}_2$ と $\mathbf{w}_2\cdot\mathbf{x}_2$ の値が入っていることが分かります．他の列についても，同様のことが成り立っています．

この例では，$X$ は $3\times 4$ で，3次元ベクトルを4つならべたものと考えられます．このとき，$Y$ は $2\times 4$ の行列となっており，その各列は，$X$の1つの列を作っている 3 次元ベクトルと2つの3次元ベクトル $\mathbf{w}_1, \mathbf{w}_2$ との内積の値を表す2次元ベクトルとなっています．
つまり，$Y$ は，行列 $X$ が表す3次元のデータ4つを，2つの3次元ベクトル $\mathbf{w}_1, \mathbf{w}_2$ を使って2次元に次元削減したものとみなせます．

上記の例では，$X$, $Y$ ともに，データが列ベクトルの形で表されています．線形代数の教科書等で線形変換（一次変換）の話が出てくるときはほとんどこの形でしょう．しかし，コンピュータでデータを扱う場合は，一つのデータが1行で表される形の方が便利です．そういう場合は，行列を転置して考えれば ok です．

In [ ]:
Xt = X.T
Xt

In [ ]:
Yt = Xt @ W.T
Yt

---
### 数物情3次元データを1次元にする例

ここからは，前に使ったことのある「数学」「物理」「情報」の点数のデータを1次元に削減する計算をやってみましょう．

In [ ]:
# 数物情データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/mpi100-mac.csv
dfMPI = pd.read_csv('mpi100-mac.csv', index_col=0)
datMPI = dfMPI.to_numpy().astype(float)
dfMPI

平均を引いたデータを作ります．

In [ ]:
# 平均
Xm = np.mean(dfMPI.to_numpy(), axis=0)
print(Xm)

# 平均を引いて NumPy 配列にする
X = dfMPI.to_numpy() - Xm
print(X.shape) # X は 100 x 3
print(X[:5, :]) # 最初の5人分を表示

一つ前の notebook で説明したように，主成分分析では，データの分散共分散行列の固有ベクトルを用いて次元削減します．今の場合は，最大固有値に対応した固有ベクトルを用いることになります．以下で，その次元削減を実行しています（固有ベクトルの計算過程は省略）．

In [ ]:
# X の分散共分散行列の最大固有値に対応する固有ベクトル
u = np.array([ 0.50399613, -0.47664448, -0.72027629])

# 1次元に次元削減
y = X @ u
for n in range(5):
    print(f'{X[n, :] + Xm}  {X[n, :]}  {y[n]:.2f}') # 最初の5人分を，元の点数，平均を引いた値，y の値の順に表示

出力された 5 つの値は，一つ前のセルの実行結果にあった5人の点数を，それぞれ1つの数値に変換したものです．

上記の例で，3次元を1次元にする次元削減ができることは分かったかもしれませんが，何をやってるのか分かりづらいですね．ベクトル `u` によって3次元の点数データがどのように変換されるか，もう少し考えてみましょう．

In [ ]:
X2 = np.array([
               [ 10,   0,   0],
               [  0,  10,   0],
               [  0,   0,  10],
               [  0,  10,  10],
               [ 10,  10,  10],
])
y = X2 @ u
for n in range(len(X2)):
    print(f'{X2[n, :]}  {y[n]:.2f}')

上記を実行すると，`X2` の各行のデータを 'u' で1次元に変換するとどうなるかが分かります．`[10, 0, 0]` というデータは，数学が平均点より10点高く，物理と情報がちょうど平均点だったことになります．このひとは，`y` の値は正ですね．一方，数学と情報が平均点で，物理のみ平均より10点高かった2番目のひとの `y` の値は負で，3番目のひとも同様となっています．また，5番目のひとは3科目すべて平均点より10点高いのですが，'y'の値は負になっています．
これらを見ると，`y` の値は，「数学が高得点で他の2科目が低得点な度合い」を表しているように思えます．

実際，3科目の点数（から平均を引いたもの）と `u` との内積の計算は，おおよそ次のような式となっています．この式を眺めると，上記の解釈に納得できる...でしょうか．
$$
y = 0.5 (\mbox{数学}) - 0.5(\mbox{物理}) - 0.7(\mbox{情報})
$$

主成分分析は，「変換後のデータの分散がなるべく大きくなるように」するのでした．そのように選ばれた `u` の方向がこのようになっているということは，この3科目の点数のデータでは，「数学の点数と他の2科目の点数の差」の分散が大きいということになります．